# Imports

In [1]:
import pandas as pd
import numpy as np
from nn_spikes import NeuralNetwork, batchTrain, test
import spike_tools, utilities, KNN_spikes
from nn_spikes import getInputsAndTargets
import plotly.express as px

In [2]:
data = pd.read_csv('./datasources/spikes/training_data.csv')
spikeLocations = pd.read_csv('./datasources/spikes/training_spike_locations.csv', index_col=0)

In [3]:
data_training, data_validation, spikeIndexes_training, spikeIndexes_validation = spike_tools.dataPreProcess(data, spikeLocations, waveformWindow=100)

3528 peaks detected.
Assigning known labels to detected spikes.
Dropped 33 detected spikes that relate to more than one label.
Returning with 3495 detected spikes.


In [5]:
data_training.loc[spikeIndexes_training]

,time (s),signal,knownSpike,knownClass,predictedSpike,predictedClass,signalSavgol,signalSavgolBP,waveform
731,0.02924,-0.121707,True,0,True,0,1.391931,0.693226,0 0.049464 1 0.025531 2 -0.04685...
905,0.03620,0.721561,False,3,True,0,0.943373,0.523216,0 0.016337 1 0.021483 2 -0.01542...
945,0.03780,0.777869,True,0,True,0,2.250108,-0.017799,0 3.056513 1 3.032812 2 2.96985...
1110,0.04440,0.243796,True,1,True,0,0.826328,0.438443,0 0.107611 1 0.106348 2 0.08650...
1412,0.05648,0.111654,True,0,True,0,1.582823,0.845603,0 0.050484 1 0.056866 2 0.06204...
...,...,...,...,...,...,...,...,...,...
1149941,45.99764,-0.302700,False,2,True,0,0.445101,0.233923,0 -0.102231 1 -0.090474 2 -0.11689...
1150458,46.01832,0.708031,False,1,True,0,0.931990,0.567270,0 0.030405 1 0.048752 2 0.03208...
1150708,46.02832,0.039795,False,2,True,0,0.213756,0.047311,0 0.013125 1 -0.050940 2 -0.03980...
1151100,46.04400,0.100155,True,1,True,0,0.812755,0.469148,0 -0.225639 1 -0.186548 2 -0.18385...


In [13]:
type(data_training.loc[731, 'waveform'])

pandas.core.series.Series

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=3)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=2)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=1)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=0)

In [ ]:
spikeInd = np.arange(0,50,3)
spikeInd

In [ ]:
duffs

In [ ]:
np.where(spikeInd == duffs[0])

In [ ]:
[np.where(spikeInd == a)[0][0] for a in duffs]

In [ ]:
mask = np.ones(len(spikeInd), dtype=bool)
mask[duffs] = False
spikeInd = spikeInd[mask]
spikeInd

# Run Neural Network Classifier

In [ ]:
results = utilities.createResultsRepo([200, 500, 700, 900, 1100])

In [ ]:
for hid in results.keys():
    
    nn = NeuralNetwork(input_nodes=len(data_training.loc[spikeIndexes_training[0], 'waveform']), 
                       hidden_nodes=int(hid), 
                       output_nodes=4, 
                       lr=0.1,
                       error_function='difference-squared')

    nn, trainingCurve, validationCurve = batchTrain(data_training=data_training,
                                                              data_validation=data_validation,
                                                              spikeIndexes_training=spikeIndexes_training, 
                                                              spikeIndexes_validation=spikeIndexes_validation, 
                                                              nn=nn,
                                                              epochs=30,
                                                              plotCurves=False)
    results[hid]['nn'] = nn
    results[hid]['trainingCurve'] = trainingCurve
    results[hid]['validationCurve'] = validationCurve

In [ ]:
utilities.plotLearningCurves(results)

In [ ]:
waveforms = data_validation[data_validation['predictedSpike']==True]['waveform']
predictionsMLP = spike_tools.classifySpikesMLP(waveforms, nn)

In [ ]:
predictionsMLP

# Run SA